In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import db_utils, sqlalchemy
import matplotlib.pyplot as plt
from matplotlib_venn import venn2, venn2_circles
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
import seaborn as sns
from scipy.stats import fisher_exact
from scipy import stats
import math
import os
import gzip
import re
import glob
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
import math

In [ ]:
# Function to count rows in a group
def count_rows(group):
    return group.shape[0]

In [2]:
# Connect to the MySQL database and read the preprocessed 3did into a DataFrame
eng = sqlalchemy.create_engine('mysql://', creator= db_utils.get_connection)
query= pd.read_sql_query('''select * from chopyan_db.3did_agg_DDI_PDB as df''', con= eng)
DDI_CS = pd.DataFrame(query)
DDI_CS

,DDI_type,PDB_ID,Chain_ID1,DomainID1,DomainName1,DomainStart1,DomainEnd1,Chain_ID2,DomainID2,DomainName2,DomainStart2,DomainEnd2,Score,Zscore,Chain1Iface,Chain2Iface,isInterchain
0,PF00001_PF00001,1gzm,A,PF00001,7tm_1,54,306,B,PF00001,7tm_1,54,306,26.18,7.82923,"E:196,E:196,E:196,E:196,E:197,N:199,E:201,E:20...","F:228,K:231,E:232,A:233,E:232,F:228,V:227,F:22...",1
1,PF00001_PF00001,1hzx,B,PF00001,7tm_1,54,246,B,PF00001,7tm_1,224,306,42.49,7.03413,"I:54,I:54,I:54,N:55,N:55,N:55,T:58,L:72,L:72,L...","V:300,P:303,V:304,A:299,V:300,P:303,Y:306,V:25...",0
2,PF00001_PF00001,1l9h,B,PF00001,7tm_1,54,246,B,PF00001,7tm_1,224,306,40.17,9.39410,"I:54,I:54,N:55,N:55,N:55,T:58,L:72,L:72,L:72,L...","V:300,P:303,A:299,V:300,P:303,Y:306,V:250,M:25...",0
3,PF00001_PF00001,2j4y,A,PF00001,7tm_1,54,306,B,PF00001,7tm_1,54,306,26.85,6.86878,"N:199,E:201,E:201,E:201,S:202,S:202,I:205,V:20...","Q:236,F:228,K:231,E:232,F:228,E:232,F:228,F:22...",1
4,PF00001_PF00001,2vt4,C,PF00001,7tm_1,58,343,D,PF00001,7tm_1,58,343,20.02,6.36751,"L:152,V:160,T:164,T:164,A:167,I:168,I:168,A:17...","F:315,A:206,A:206,A:210,W:181,A:210,I:214,W:18...",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235443,PF18841_PF17433,1wmr,A,PF18841,B_solenoid_dext,321,354,A,PF17433,Glyco_hydro_49N,19,184,9.52,4.52235,"D:341,D:341,D:341,K:343,K:343,V:345,V:345,V:34...","R:153,S:182,P:183,S:101,S:182,Q:97,W:99,L:178,...",0
235444,PF18841_PF17433,1x0c,B,PF18841,B_solenoid_dext,321,354,B,PF17433,Glyco_hydro_49N,19,184,9.52,4.45403,"D:341,D:341,D:341,K:343,K:343,V:345,V:345,V:34...","R:153,S:182,P:183,S:101,S:182,Q:97,W:99,L:178,...",0
235445,PF18841_PF17433,2z8g,A,PF18841,B_solenoid_dext,321,354,A,PF17433,Glyco_hydro_49N,19,184,9.24,4.49346,"D:341,D:341,D:341,K:343,K:343,V:345,V:345,V:34...","R:153,S:182,P:183,S:101,S:182,Q:97,W:99,L:178,...",0
235446,PF18841_PF17433,3wwg,C,PF18841,B_solenoid_dext,321,354,C,PF17433,Glyco_hydro_49N,19,184,9.24,4.61412,"D:341,D:341,D:341,K:343,K:343,V:345,V:345,V:34...","R:153,S:182,P:183,S:101,S:182,Q:97,W:99,L:178,...",0


In [3]:
DDI_CS['PDB_ID']= DDI_CS['PDB_ID'].str.upper()

In [4]:
# Add another column counting residue-residue contactcs of an interface,
# check whether the # ofcontacts are the same for both domains
contacts1 = []
for i in DDI_CS['Chain1Iface']:
    contacts1.append(len(i.split(',')))
DDI_CS['contacts1'] = contacts1
contacts2 = []
for i in DDI_CS['Chain2Iface']:
    contacts2.append(len(i.split(',')))
DDI_CS['contacts2'] = contacts2
DDI_CS[(DDI_CS['contacts1'] != DDI_CS['contacts2'])]

,DDI_type,PDB_ID,Chain_ID1,DomainID1,DomainName1,DomainStart1,DomainEnd1,Chain_ID2,DomainID2,DomainName2,DomainStart2,DomainEnd2,Score,Zscore,Chain1Iface,Chain2Iface,isInterchain,contacts1,contacts2


In [5]:
DDI_CS = DDI_CS.drop(columns= ['contacts2'])

In [6]:
# Merge DDI_CS with additional protein information
df_protein_info = pd.read_csv('/Users/johgeist/Downloads/DDI_type_structure_level_interchain_homo_protein_whole_3did.csv', index_col=0)
df_protein_info= df_protein_info.drop(columns=['IsInterchain', 'Chain1', 'Chain2'])
DDI_CS= pd.merge(DDI_CS, df_protein_info, on=['DDI_type', 'PDB_ID'])

In [8]:
# Drop rows that relate to intrachain or homodimeric interfaces
DDI_inter= DDI_CS.drop(index= DDI_CS[DDI_CS['isInterchain'] == 0].index)
DDI_inter_hetero = DDI_inter.drop(index= DDI_inter[DDI_inter['HomoProtein'] == 1].index)

In [9]:
DDI_inter_hetero.to_csv('/Users/johgeist/Downloads/3did_project_dataframes_new/3did_metrics_inter_hetero_subset_structure_level.csv', index=False)

In [10]:
# Select one interface per DDI type as well as its metrics based on the highest 3did score
df_index_max_Score = DDI_inter_hetero.groupby('DDI_type').agg({'Score': 'idxmax'})
DDI_inter_hetero_grouped_max_Score = DDI_inter_hetero[DDI_inter_hetero.index.isin(df_index_max_Score.Score.tolist())]

In [11]:
DDI_inter_hetero_grouped_max_Score

,DDI_type,PDB_ID,Chain_ID1,DomainID1,DomainName1,DomainStart1,DomainEnd1,Chain_ID2,DomainID2,DomainName2,...,Score,Zscore,Chain1Iface,Chain2Iface,isInterchain,contacts1,HomoDDI,UniProt1,UniProt2,HomoProtein
72,PF00001_PF00048,5UIW,A,PF00001,7tm_1,47,297,B,PF00048,IL8,...,13.26,5.101720,"Y:89,Q:170,Q:170,K:171,K:171,E:172,E:172,E:172...","Q:8,F:28,Y:29,E:26,F:28,Q:8,S:9,F:28,S:31,Q:48...",1,51,0,"P51681,P00268",P13501,0
82,PF00001_PF00085,6H7O,B,PF00001,7tm_1,58,343,F,PF00085,Thioredoxin,...,1.57,1.963520,"V:103,V:103,R:104,R:104,R:104,R:104,R:104","G:51,K:52,D:47,E:48,Y:49,Q:50,G:51",1,7,0,P07700,P0AA25,0
84,PF00001_PF00087,6WJC,A,PF00001,7tm_1,42,418,C,PF00087,Toxin_TOLIP,...,15.76,5.828060,"Y:82,W:164,W:164,L:167,L:167,V:168,V:168,V:168...","R:34,I:9,W:10,W:10,F:11,I:9,W:10,F:11,P:12,R:4...",1,44,0,"P11229,D9IEF7",Q8QGR0,0
85,PF00001_PF00322,5GLH,A,PF00001,7tm_1,107,387,B,PF00322,Endothelin,...,31.19,7.365980,"I:157,N:158,N:158,N:158,K:161,K:161,K:161,K:16...","I:20,I:19,I:20,W:21,C:15,H:16,L:17,D:18,I:19,H...",1,78,0,"P00720,P24530",P05305,0
92,PF00001_PF00339,6TKO,A,PF00001,7tm_1,58,343,B,PF00339,Arrestin_N,...,18.24,5.657100,"R:71,Q:73,T:74,L:75,L:75,T:76,N:77,I:80,R:139,...","D:67,T:136,T:136,E:66,T:136,E:66,E:66,V:70,E:6...",1,38,0,P07700,P49407,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235247,PF18782_PF00559,6NIL,A,PF18782,NAD2,193,372,C,PF00559,Vif,...,7.74,3.639300,"R:252,L:255,S:256,S:256,C:259,D:260,D:260,D:26...","P:49,W:70,H:43,W:70,Q:83,R:15,N:19,K:22,Q:83,R...",1,23,0,Q8IUX4,P12504,0
235281,PF18784_PF00071,3NBZ,D,PF18784,CRM1_repeat_2,405,472,F,PF00071,Ras,...,8.76,4.306780,"E:429,L:431,L:431,L:431,V:433,E:434,D:436,D:43...","N:156,S:153,N:154,Y:155,S:153,R:29,K:37,K:152,...",1,25,0,Q6P5F9,P62826,0
235371,PF18801_PF00072,3Q15,A,PF18801,RapH_N,4,65,C,PF00072,Response_reg,...,10.48,4.961570,"E:45,D:46,Q:47,Q:47,Q:47,Q:47,D:48,L:50,L:50,L...","Y:84,Y:84,Q:12,K:56,A:83,Y:84,Q:12,A:83,Y:84,K...",1,22,0,Q59HN8,P06628,0
235372,PF18806_PF00071,2X19,B,PF18806,Importin_rep_3,826,900,A,PF00071,Ras,...,0.05,0.237612,"P:829,E:830,E:830,E:830,K:835,S:880,S:880","Y:41,K:40,Y:41,I:42,T:95,S:96,I:98",1,7,0,O94829,P32835,0


In [12]:
# Check whether there are NaN values for the UniProt IDs (from the protein information df)
DDI_inter_hetero_grouped_max_Score[(DDI_inter_hetero_grouped_max_Score['UniProt1']== 'No UniProt') & (DDI_inter_hetero_grouped_max_Score['UniProt2']== 'No UniProt')]

,DDI_type,PDB_ID,Chain_ID1,DomainID1,DomainName1,DomainStart1,DomainEnd1,Chain_ID2,DomainID2,DomainName2,...,Score,Zscore,Chain1Iface,Chain2Iface,isInterchain,contacts1,HomoDDI,UniProt1,UniProt2,HomoProtein


In [13]:
# Keep only those columns that are necessary for further analysis, DDI_type is the unique idendifier for each DDI
# PDB ID of the structure with the highest 3did score is needed later for merging with ProtCID data
DDI_inter_hetero_grouped_max_Score = DDI_inter_hetero_grouped_max_Score.filter(['DDI_type', 'Score', 'Zscore', 'contacts1', 'PDB_ID'])

In [14]:
DDI_inter_hetero_grouped_max_Score

,DDI_type,Score,Zscore,contacts1,PDB_ID
72,PF00001_PF00048,13.26,5.101720,51,5UIW
82,PF00001_PF00085,1.57,1.963520,7,6H7O
84,PF00001_PF00087,15.76,5.828060,44,6WJC
85,PF00001_PF00322,31.19,7.365980,78,5GLH
92,PF00001_PF00339,18.24,5.657100,38,6TKO
...,...,...,...,...,...
235247,PF18782_PF00559,7.74,3.639300,23,6NIL
235281,PF18784_PF00071,8.76,4.306780,25,3NBZ
235371,PF18801_PF00072,10.48,4.961570,22,3Q15
235372,PF18806_PF00071,0.05,0.237612,7,2X19


In [15]:
DDI_inter_hetero_grouped_max_Score.rename({'PDB_ID': 'PDB_ID_max'}, inplace=True, axis=1)

In [16]:
# DDI_merged combines DDI_inter_hetero_grouped_max_Score with additional data from DDI_CS, that can only be inferred 
# from there (# inter-/ intrachain structures etc.) or aid further annoattaion of the DDI
DDI_merged = pd.merge(DDI_CS.groupby('DDI_type').agg({'DomainID1': 'first','DomainID2': 'first',
                                                      'DomainName1': 'first','DomainName2': 'first',
                                                      'PDB_ID': count_rows,
                                                      'isInterchain': sum}).reset_index(),
                                                          DDI_inter_hetero_grouped_max_Score,
                                                        on='DDI_type', how= 'inner')

In [17]:
DDI_merged.rename({'PDB_ID': 'Num_structures', 'isInterchain': 'Num_interchain_structures',
                   'HomoProtein': 'Num_homo_protein_structures', 'HomoDDI': 'Homo_DDI'}, inplace=True, axis=1)
DDI_merged['Interchain_fraction'] = DDI_merged['Num_interchain_structures']/DDI_merged['Num_structures']
DDI_merged['Num_intrachain_structures']= DDI_merged['Num_structures']-DDI_merged['Num_interchain_structures']

In [18]:
DDI_merged

,DDI_type,DomainID1,DomainID2,DomainName1,DomainName2,Num_structures,Num_interchain_structures,Score,Zscore,contacts1,PDB_ID_max,Interchain_fraction,Num_intrachain_structures
0,PF00001_PF00048,PF00001,PF00048,7tm_1,IL8,8,8,13.26,5.101720,51,5UIW,1.000000,0
1,PF00001_PF00085,PF00001,PF00085,7tm_1,Thioredoxin,7,6,1.57,1.963520,7,6H7O,0.857143,1
2,PF00001_PF00087,PF00001,PF00087,7tm_1,Toxin_TOLIP,1,1,15.76,5.828060,44,6WJC,1.000000,0
3,PF00001_PF00322,PF00001,PF00322,7tm_1,Endothelin,4,4,31.19,7.365980,78,5GLH,1.000000,0
4,PF00001_PF00339,PF00001,PF00339,7tm_1,Arrestin_N,6,3,18.24,5.657100,38,6TKO,0.500000,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5719,PF18782_PF00559,PF18782,PF00559,NAD2,Vif,1,1,7.74,3.639300,23,6NIL,1.000000,0
5720,PF18784_PF00071,PF18784,PF00071,CRM1_repeat_2,Ras,34,34,8.76,4.306780,25,3NBZ,1.000000,0
5721,PF18801_PF00072,PF18801,PF00072,RapH_N,Response_reg,1,1,10.48,4.961570,22,3Q15,1.000000,0
5722,PF18806_PF00071,PF18806,PF00071,Importin_rep_3,Ras,1,1,0.05,0.237612,7,2X19,1.000000,0


In [19]:
DDI_merged.to_csv('/Users/johgeist/Downloads/3did_project_dataframes_new/3did_metrics_inter_hetero_subset.csv', index=False)

In [20]:
# Connect to the MySQL database and read the interactome cooccurrence data into a DataFrame, that was generated before
eng = sqlalchemy.create_engine('mysql://', creator= db_utils.get_connection)
query= pd.read_sql_query('''select * from chopyan_db.DDI_type_features as df''', con= eng)
DDI_type_feat= pd.DataFrame(query)
DDI_type_feat

,DDI_type,DDI_type_sorted,PfamID1,DomainID1,DomainName1,PfamID2,DomainID2,DomainName2,HuRIpvalue,HuRIzscore,BioPlexpvalue,BioPlexzscore,StructureScore,DDI_type_score,ManuallyCurated,homoDDI,hasInterchain
0,PF10417_PF10417,PF10417_PF10417,PF10417,1-cysPrx_C,C-terminal domain of 1-Cys peroxiredoxin,PF10417,1-cysPrx_C,C-terminal domain of 1-Cys peroxiredoxin,0.000,17.704923,0.000,63.245561,None,NaN,0,1,1
1,PF10417_PF00578,PF00578_PF10417,PF10417,1-cysPrx_C,C-terminal domain of 1-Cys peroxiredoxin,PF00578,AhpC-TSA,AhpC/TSA family,0.000,17.704923,0.000,63.245561,None,0.646112,0,0,1
2,PF10417_PF02195,PF02195_PF10417,PF10417,1-cysPrx_C,C-terminal domain of 1-Cys peroxiredoxin,PF02195,ParBc,ParB-like nuclease domain,NaN,NaN,1.000,-0.031639,None,0.896050,0,0,1
3,PF10417_PF00085,PF00085_PF10417,PF10417,1-cysPrx_C,C-terminal domain of 1-Cys peroxiredoxin,PF00085,Thioredoxin,Thioredoxin,0.039,3.293199,0.035,5.250850,None,0.254769,0,0,1
4,PF12574_PF12574,PF12574_PF12574,PF12574,120_Rick_ant,120 KDa Rickettsia surface antigen,PF12574,120_Rick_ant,120 KDa Rickettsia surface antigen,NaN,NaN,NaN,NaN,None,NaN,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14967,PF05707_PF05707,PF05707_PF05707,PF05707,Zot,Zonular occludens toxin (Zot),PF05707,Zot,Zonular occludens toxin (Zot),NaN,NaN,NaN,NaN,None,NaN,0,1,1
14968,PF16916_PF16916,PF16916_PF16916,PF16916,ZT_dimer,Dimerisation domain of Zinc Transporter,PF16916,ZT_dimer,Dimerisation domain of Zinc Transporter,NaN,NaN,NaN,NaN,None,NaN,0,1,1
14969,PF00791_PF00791,PF00791_PF00791,PF00791,ZU5,ZU5 domain,PF00791,ZU5,ZU5 domain,1.000,-0.100504,1.000,-0.110208,None,NaN,0,1,1
14970,PF09817_PF09817,PF09817_PF09817,PF09817,Zwilch,"RZZ complex, subunit zwilch",PF09817,Zwilch,"RZZ complex, subunit zwilch",NaN,NaN,1.000,0.000000,None,NaN,0,1,1


In [21]:
feat= DDI_type_feat.filter(['DDI_type', 'HuRIzscore', 'BioPlexzscore'])
feat

,DDI_type,HuRIzscore,BioPlexzscore
0,PF10417_PF10417,17.704923,63.245561
1,PF10417_PF00578,17.704923,63.245561
2,PF10417_PF02195,NaN,-0.031639
3,PF10417_PF00085,3.293199,5.250850
4,PF12574_PF12574,NaN,NaN
...,...,...,...
14967,PF05707_PF05707,NaN,NaN
14968,PF16916_PF16916,NaN,NaN
14969,PF00791_PF00791,-0.100504,-0.110208
14970,PF09817_PF09817,NaN,0.000000


In [22]:
# Merge the wanted cooccurrence features to the DDI_merged dataframe
df_3did_HuRI_BioPlex = pd.merge(DDI_merged, feat, how= 'inner')

In [23]:
df_3did_HuRI_BioPlex

,DDI_type,DomainID1,DomainID2,DomainName1,DomainName2,Num_structures,Num_interchain_structures,Score,Zscore,contacts1,PDB_ID_max,Interchain_fraction,Num_intrachain_structures,HuRIzscore,BioPlexzscore
0,PF00001_PF00048,PF00001,PF00048,7tm_1,IL8,8,8,13.26,5.101720,51,5UIW,1.000000,0,8.015862,-2.337582
1,PF00001_PF00085,PF00001,PF00085,7tm_1,Thioredoxin,7,6,1.57,1.963520,7,6H7O,0.857143,1,-0.455672,-1.268307
2,PF00001_PF00087,PF00001,PF00087,7tm_1,Toxin_TOLIP,1,1,15.76,5.828060,44,6WJC,1.000000,0,-0.077693,NaN
3,PF00001_PF00322,PF00001,PF00322,7tm_1,Endothelin,4,4,31.19,7.365980,78,5GLH,1.000000,0,10.129397,-0.495000
4,PF00001_PF00339,PF00001,PF00339,7tm_1,Arrestin_N,6,3,18.24,5.657100,38,6TKO,0.500000,3,12.478734,1.637844
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5719,PF18782_PF00559,PF18782,PF00559,NAD2,Vif,1,1,7.74,3.639300,23,6NIL,1.000000,0,NaN,NaN
5720,PF18784_PF00071,PF18784,PF00071,CRM1_repeat_2,Ras,34,34,8.76,4.306780,25,3NBZ,1.000000,0,-0.309397,-0.216169
5721,PF18801_PF00072,PF18801,PF00072,RapH_N,Response_reg,1,1,10.48,4.961570,22,3Q15,1.000000,0,NaN,NaN
5722,PF18806_PF00071,PF18806,PF00071,Importin_rep_3,Ras,1,1,0.05,0.237612,7,2X19,1.000000,0,2.846743,-0.351625


In [24]:
df_3did_HuRI_BioPlex.to_csv('/Users/johgeist/Downloads/3did_project_dataframes_new/3did_Interactome_metrics_inter_hetero_subset.csv', index=False)

In [25]:
# Read in ProtCID cluster information as a DataFrame, that was generated seperately before
# Only those clusters are present for a given Pfam-Pfam combination, that include the PDB ID with the highest
# score in 3did
df_counts = pd.read_csv('/Users/johgeist/Downloads/3did_project_dataframes_new/ProtCID_parsed_files/Master_counts_df_ProtCID_20231017.csv')

In [26]:
df_counts

,DomainNames,FileName,DDI_type,ClusterID,SurfaceArea,MinSeqIdentity,MediumSurfaceArea,PDB_ID_max,UnpCode,PdbID,CFID
0,Med21 Med4,(Med21)(Med4)_19935.txt,PF11221_PF10018,1,481.3500,27.0,506.75,5n9j,2,2,2
1,PI3Ka SH2,(PI3Ka)(SH2)_13501.txt,PF00613_PF00017,1,382.7833,97.0,347.40,5swr,1,30,5
2,Pox_Rap94 RNA_pol_Rpb2_7,(Pox_Rap94)(RNA_pol_Rpb2_7)_35453.txt,PF03294_PF04560,1,480.2750,99.0,527.35,6rfl,2,2,2
3,Adeno_hexon_C Adeno_PVIII,(Adeno_hexon_C)(Adeno_PVIII)_1493.txt,PF01310_PF03678,1,396.3250,33.0,408.70,6cgv,6,7,7
4,Adeno_hexon_C Adeno_PVIII,(Adeno_hexon_C)(Adeno_PVIII)_1493.txt,PF01310_PF03678,2,298.4818,25.0,374.75,6cgv,5,6,6
...,...,...,...,...,...,...,...,...,...,...,...
4343,Fer4_7 NADH_Oxid_Nqo15,(Fer4_7)(NADH_Oxid_Nqo15)_8080.txt,PF12838_PF11497,1,323.8731,99.0,326.20,4hea,1,14,8
4344,Chordopox_RPO7 Pox_RNA_pol_35,(Chordopox_RPO7)(Pox_RNA_pol_35)_35486.txt,PF05864_PF03396,1,1492.7250,98.0,1503.20,6rie,2,4,4
4345,ig MHC_I,(ig)(MHC_I)_18510.txt,PF00129_PF00047,1,311.3024,79.0,297.50,5t70,9,13,12
4346,ig MHC_I,(ig)(MHC_I)_18510.txt,PF00129_PF00047,2,243.0559,30.0,231.25,5t70,7,11,10


In [27]:
# Metrics were grouped on DDI level selecting the maximum (minimum) value among those clusters of a given Pfam-Pfam
# combination
df_counts_grouped_max_or = df_counts.groupby('DDI_type').agg({'CFID': max, 'MinSeqIdentity': min,
                                                              'UnpCode': max, 'ClusterID': max})
df_counts_grouped_max_or = df_counts_grouped_max_or.rename(columns ={'CFID': 'Num crystal forms',
                                                                      'MinSeqIdentity': 'Min seq identity',
                                                                     'UnpCode': 'Num of proteins', 
                                                                      'ClusterID': 'Num clusters'})
df_counts_grouped_max_or.reset_index(inplace=True)

In [28]:
# Merge the df_3did_HuRI_BioPlex with the grouped cluster information from ProtCID
df_3did_HuRI_BioPlex['DDI_type']=df_3did_HuRI_BioPlex['DDI_type'].astype('str')
df_counts_grouped_max_or['DDI_type']=df_counts_grouped_max_or['DDI_type'].astype('str')
df_3did_HuRI_BioPlex_ProtCID = df_3did_HuRI_BioPlex.merge(df_counts_grouped_max_or,on= 'DDI_type', how= 'left')

In [30]:
# check for how many DDI types in the selected 3did subset there is no ProtCID cluster available
df_3did_HuRI_BioPlex_ProtCID[df_3did_HuRI_BioPlex_ProtCID['Num crystal forms'].isna()]

,DDI_type,DomainID1,DomainID2,DomainName1,DomainName2,Num_structures,Num_interchain_structures,Score,Zscore,contacts1,PDB_ID_max,Interchain_fraction,Num_intrachain_structures,HuRIzscore,BioPlexzscore,Num crystal forms,Min seq identity,Num of proteins,Num clusters
2,PF00001_PF00087,PF00001,PF00087,7tm_1,Toxin_TOLIP,1,1,15.76,5.82806,44,6WJC,1.0,0,-0.077693,NaN,NaN,NaN,NaN,NaN
5,PF00001_PF00400,PF00001,PF00400,7tm_1,WD40,2,2,2.95,2.73640,5,6OSA,1.0,0,0.456522,-4.290103,NaN,NaN,NaN,NaN
16,PF00002_PF04901,PF00002,PF04901,7tm_2,RAMP,4,4,29.41,7.42986,59,6UUN,1.0,0,7.604178,-0.600940,NaN,NaN,NaN,NaN
20,PF00004_PF00101,PF00004,PF00101,AAA,RuBisCO_small,1,1,5.34,3.32008,16,6Z1F,1.0,0,NaN,NaN,NaN,NaN,NaN,NaN
21,PF00004_PF00227,PF00004,PF00227,AAA,Proteasome,5,5,2.37,2.35372,10,1E94,1.0,0,2.041022,10.089070,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5716,PF18773_PF02792,PF18773,PF02792,Importin_rep,Mago_nashi,1,1,5.61,4.50071,10,2X1G,1.0,0,10.493384,0.000000,NaN,NaN,NaN,NaN
5718,PF18779_PF00074,PF18779,PF00074,LRR_RI_capping,RnaseA,1,1,6.45,4.17083,11,2BEX,1.0,0,36.514878,30.811234,NaN,NaN,NaN,NaN
5719,PF18782_PF00559,PF18782,PF00559,NAD2,Vif,1,1,7.74,3.63930,23,6NIL,1.0,0,NaN,NaN,NaN,NaN,NaN,NaN
5721,PF18801_PF00072,PF18801,PF00072,RapH_N,Response_reg,1,1,10.48,4.96157,22,3Q15,1.0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
# Impute missing values for ProtCID metrics for those DDIs that do not have a cluster in ProtCID
df_3did_HuRI_BioPlex_ProtCID['Num clusters'].fillna(0, inplace=True)
df_3did_HuRI_BioPlex_ProtCID['Min seq identity'].fillna(100, inplace=True)
df_3did_HuRI_BioPlex_ProtCID['Num crystal forms'].fillna(1, inplace=True)
df_3did_HuRI_BioPlex_ProtCID['Num of proteins'].fillna(1, inplace=True)

In [32]:
df_3did_HuRI_BioPlex_ProtCID

,DDI_type,DomainID1,DomainID2,DomainName1,DomainName2,Num_structures,Num_interchain_structures,Score,Zscore,contacts1,PDB_ID_max,Interchain_fraction,Num_intrachain_structures,HuRIzscore,BioPlexzscore,Num crystal forms,Min seq identity,Num of proteins,Num clusters
0,PF00001_PF00048,PF00001,PF00048,7tm_1,IL8,8,8,13.26,5.101720,51,5UIW,1.000000,0,8.015862,-2.337582,5.0,18.0,4.0,1.0
1,PF00001_PF00085,PF00001,PF00085,7tm_1,Thioredoxin,7,6,1.57,1.963520,7,6H7O,0.857143,1,-0.455672,-1.268307,2.0,95.0,2.0,4.0
2,PF00001_PF00087,PF00001,PF00087,7tm_1,Toxin_TOLIP,1,1,15.76,5.828060,44,6WJC,1.000000,0,-0.077693,NaN,1.0,100.0,1.0,0.0
3,PF00001_PF00322,PF00001,PF00322,7tm_1,Endothelin,4,4,31.19,7.365980,78,5GLH,1.000000,0,10.129397,-0.495000,3.0,57.0,3.0,1.0
4,PF00001_PF00339,PF00001,PF00339,7tm_1,Arrestin_N,6,3,18.24,5.657100,38,6TKO,0.500000,3,12.478734,1.637844,5.0,18.0,4.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5719,PF18782_PF00559,PF18782,PF00559,NAD2,Vif,1,1,7.74,3.639300,23,6NIL,1.000000,0,NaN,NaN,1.0,100.0,1.0,0.0
5720,PF18784_PF00071,PF18784,PF00071,CRM1_repeat_2,Ras,34,34,8.76,4.306780,25,3NBZ,1.000000,0,-0.309397,-0.216169,10.0,50.0,9.0,1.0
5721,PF18801_PF00072,PF18801,PF00072,RapH_N,Response_reg,1,1,10.48,4.961570,22,3Q15,1.000000,0,NaN,NaN,1.0,100.0,1.0,0.0
5722,PF18806_PF00071,PF18806,PF00071,Importin_rep_3,Ras,1,1,0.05,0.237612,7,2X19,1.000000,0,2.846743,-0.351625,2.0,22.0,2.0,1.0


In [34]:
df_3did_HuRI_BioPlex_ProtCID.to_csv('/Users/johgeist/Downloads/3did_project_dataframes_new/3did_Interactome_ProtCID_metrics_inter_hetero_subset.csv', index=False)

In [36]:
# Create 3did subset for representativity analysis done seperately
# Generate new 3did subset without synthetic construct domains Ank PF00023 (14), PF12796(48),
# PF13606(1), PF13637(15), PF13657(1), V-set PF07686 (79)
syn_cons_domains = ['PF00023', 'PF07686', 'PF12796', 'PF13606', 'PF13637', 'PF13657']
final_subset = df_3did_HuRI_BioPlex_ProtCID.copy()
final_subset['DDI_type'] = final_subset['DDI_type'].str.split('_')
rows_to_drop = final_subset.apply(lambda row: any(set(syn_cons_domains).intersection(row['DDI_type'])), axis=1)
final_subset = final_subset[~rows_to_drop]

In [37]:
final_subset

,DDI_type,DomainID1,DomainID2,DomainName1,DomainName2,Num_structures,Num_interchain_structures,Score,Zscore,contacts1,PDB_ID_max,Interchain_fraction,Num_intrachain_structures,HuRIzscore,BioPlexzscore,Num crystal forms,Min seq identity,Num of proteins,Num clusters
0,"[PF00001, PF00048]",PF00001,PF00048,7tm_1,IL8,8,8,13.26,5.101720,51,5UIW,1.000000,0,8.015862,-2.337582,5.0,18.0,4.0,1.0
1,"[PF00001, PF00085]",PF00001,PF00085,7tm_1,Thioredoxin,7,6,1.57,1.963520,7,6H7O,0.857143,1,-0.455672,-1.268307,2.0,95.0,2.0,4.0
2,"[PF00001, PF00087]",PF00001,PF00087,7tm_1,Toxin_TOLIP,1,1,15.76,5.828060,44,6WJC,1.000000,0,-0.077693,NaN,1.0,100.0,1.0,0.0
3,"[PF00001, PF00322]",PF00001,PF00322,7tm_1,Endothelin,4,4,31.19,7.365980,78,5GLH,1.000000,0,10.129397,-0.495000,3.0,57.0,3.0,1.0
4,"[PF00001, PF00339]",PF00001,PF00339,7tm_1,Arrestin_N,6,3,18.24,5.657100,38,6TKO,0.500000,3,12.478734,1.637844,5.0,18.0,4.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5719,"[PF18782, PF00559]",PF18782,PF00559,NAD2,Vif,1,1,7.74,3.639300,23,6NIL,1.000000,0,NaN,NaN,1.0,100.0,1.0,0.0
5720,"[PF18784, PF00071]",PF18784,PF00071,CRM1_repeat_2,Ras,34,34,8.76,4.306780,25,3NBZ,1.000000,0,-0.309397,-0.216169,10.0,50.0,9.0,1.0
5721,"[PF18801, PF00072]",PF18801,PF00072,RapH_N,Response_reg,1,1,10.48,4.961570,22,3Q15,1.000000,0,NaN,NaN,1.0,100.0,1.0,0.0
5722,"[PF18806, PF00071]",PF18806,PF00071,Importin_rep_3,Ras,1,1,0.05,0.237612,7,2X19,1.000000,0,2.846743,-0.351625,2.0,22.0,2.0,1.0


In [38]:
final_subset['DDI_type'] = final_subset['DDI_type'].str.join('_')

In [39]:
final_subset

,DDI_type,DomainID1,DomainID2,DomainName1,DomainName2,Num_structures,Num_interchain_structures,Score,Zscore,contacts1,PDB_ID_max,Interchain_fraction,Num_intrachain_structures,HuRIzscore,BioPlexzscore,Num crystal forms,Min seq identity,Num of proteins,Num clusters
0,PF00001_PF00048,PF00001,PF00048,7tm_1,IL8,8,8,13.26,5.101720,51,5UIW,1.000000,0,8.015862,-2.337582,5.0,18.0,4.0,1.0
1,PF00001_PF00085,PF00001,PF00085,7tm_1,Thioredoxin,7,6,1.57,1.963520,7,6H7O,0.857143,1,-0.455672,-1.268307,2.0,95.0,2.0,4.0
2,PF00001_PF00087,PF00001,PF00087,7tm_1,Toxin_TOLIP,1,1,15.76,5.828060,44,6WJC,1.000000,0,-0.077693,NaN,1.0,100.0,1.0,0.0
3,PF00001_PF00322,PF00001,PF00322,7tm_1,Endothelin,4,4,31.19,7.365980,78,5GLH,1.000000,0,10.129397,-0.495000,3.0,57.0,3.0,1.0
4,PF00001_PF00339,PF00001,PF00339,7tm_1,Arrestin_N,6,3,18.24,5.657100,38,6TKO,0.500000,3,12.478734,1.637844,5.0,18.0,4.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5719,PF18782_PF00559,PF18782,PF00559,NAD2,Vif,1,1,7.74,3.639300,23,6NIL,1.000000,0,NaN,NaN,1.0,100.0,1.0,0.0
5720,PF18784_PF00071,PF18784,PF00071,CRM1_repeat_2,Ras,34,34,8.76,4.306780,25,3NBZ,1.000000,0,-0.309397,-0.216169,10.0,50.0,9.0,1.0
5721,PF18801_PF00072,PF18801,PF00072,RapH_N,Response_reg,1,1,10.48,4.961570,22,3Q15,1.000000,0,NaN,NaN,1.0,100.0,1.0,0.0
5722,PF18806_PF00071,PF18806,PF00071,Importin_rep_3,Ras,1,1,0.05,0.237612,7,2X19,1.000000,0,2.846743,-0.351625,2.0,22.0,2.0,1.0


In [40]:
# Count how many DDI types are including domains associated with synthetic constructs
# 454 occurrences of synthetic construct domains in DDI_types in the 3did subset
len(df_3did_HuRI_BioPlex_ProtCID) - len(final_subset)

454

In [41]:
final_subset.to_csv('/Users/johgeist/Downloads/3did_project_dataframes_new/3did_Interactome_ProtCID_metrics_inter_hetero_subset_without_synthetic_constructs.csv', index=False)

In [42]:
# Read in manual curation dataset as a DataFrame and merge with the original 3did DataFrame (DDI_CS) to get the
# respective 3did score for all interfaces 
# select the rows relating to the structure with the interface with the highest 3did score (same as before)
# for a Pfam-Pfam combination
df_mc = pd.read_csv('/Users/johgeist/Downloads/DDI_curation table_wihtout_homo_protein_structures_4.csv')
df_mc = df_mc[~df_mc['DDI_type'].str.contains('curation')]
df_mc['PDB_ID'] = df_mc['PDB_ID'].str.upper()

In [43]:
df_Score = pd.merge(DDI_CS, df_mc, on= ['DDI_type', 'PDB_ID'], how = 'inner')
df_Score

,DDI_type,PDB_ID,Chain_ID1,DomainID1,DomainName1,DomainStart1,DomainEnd1,Chain_ID2,DomainID2,DomainName2,...,ProtCID_cluster_approved,Motif_binding_domain_present,Motif_binding_domain,DMI_overlap_checked,DMI_overlap_identified,DDI_approved,Curator,Comments,non-approved category,Unnamed: 19
0,PF00004_PF10584,6EPF,I,PF00004,AAA,222,355,C,PF10584,Proteasome_A_N,...,1.0,0.0,NaN,NaN,NaN,0.0,"JG, KL",1 out of 1 supporting CS showed an interchain ...,HMM (at least one Pfam has unfolded structure),linear stretch within domain
1,PF00009_PF01873,2D74,A,PF00009,GTP_EFTU,9,201,B,PF01873,eIF-5_eIF-2B,...,0.0,0.0,NaN,NaN,NaN,1.0,JG,11 out of 11 supporting CS showed interchain i...,NaN,NaN
2,PF00010_PF02344,1NKP,B,PF00010,HLH,203,254,D,PF02344,Myc-LZ,...,0.0,0.0,NaN,NaN,NaN,0.0,JG,Second structure checked for PF00010_PF02344. ...,NaN,NaN
3,PF00010_PF02344,6G6L,B,PF00010,HLH,214,254,C,PF02344,Myc-LZ,...,0.0,0.0,NaN,NaN,NaN,0.0,"JG, KL",2 out of 6 supporting structures in 3did showe...,crystal contact,NaN
4,PF00019_PF00041,4UI2,B,PF00019,TGF_beta,294,395,A,PF00041,fn3,...,NaN,0.0,NaN,NaN,NaN,0.0,JG,Only 1 supporting CS was present for the DDI t...,crystal contact,study describes crystal contact interface
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,PF17292_PF08644,4KHB,H,PF17292,POB3_N,5,97,G,PF08644,SPT16,...,1.0,0.0,NaN,NaN,NaN,0.0,"JG, KL",3 out of 3 supporting CS displayed interchain ...,contact is minimal,NaN
91,PF17292_PF08644,6UPK,H,PF17292,POB3_N,5,99,G,PF08644,SPT16,...,1.0,0.0,NaN,NaN,NaN,0.0,JG,Second structure checked for PF17292_PF08644. ...,NaN,NaN
92,PF17820_PF16523,3L4F,D,PF17820,PDZ_6,701,755,A,PF16523,betaPIX_CC,...,NaN,0.0,NaN,NaN,NaN,0.0,"JG, JL",Only one structure supporting the DDI type (Sc...,HMM (at least one Pfam has unfolded structure),motif is C-terminally included in the Pfam match
93,PF17838_PF00071,3KZ1,B,PF17838,PH_16,950,1079,E,PF00071,Ras,...,0.0,0.0,NaN,NaN,NaN,1.0,JG,9 out of 9 supporting CS displayed interchain ...,NaN,NaN


In [44]:
df_Score_index = df_Score.groupby('DDI_type').agg({'Score': 'idxmax'})
df_Score_max = df_Score[df_Score.index.isin(df_Score_index.Score.tolist())]

In [45]:
df_Score_max = df_Score_max.filter(['DDI_type', 'Score', 'PDB_ID'])
df_Score_max

,DDI_type,Score,PDB_ID
0,PF00004_PF10584,3.28,6EPF
1,PF00009_PF01873,9.54,2D74
3,PF00010_PF02344,3.30,6G6L
4,PF00019_PF00041,4.56,4UI2
5,PF00023_PF07686,1.25,4NIK
...,...,...,...
88,PF16004_PF00400,9.03,6ID0
90,PF17292_PF08644,2.26,4KHB
92,PF17820_PF16523,3.24,3L4F
93,PF17838_PF00071,14.92,3KZ1


In [46]:
# Connect to MySQL and load the results of the AlphaFold Multimer predictions into a DataFrame
eng = sqlalchemy.create_engine('mysql://', creator= db_utils.get_connection)
query = pd.read_sql_query("""select * from chopyan_db.AlphaFold_prediction_result where project_name = 'DDI_predictor' and run_id = 'run5';""", con=eng)
df_AF= pd.DataFrame(query)
df_AF = df_AF.rename({'ddi_name': 'DDI_type'}, axis=1)
df_AF

,project_name,prediction_name,run_id,chainA_length,chainB_length,model_id,model_confidence,chainA_intf_avg_plddt,chainB_intf_avg_plddt,intf_avg_plddt,...,num_align_atoms_domain,align_score_domain,num_align_resi_domain,RMSD_backbone_peptide,RMSD_all_atom_peptide,known_motif_plddt,DockQ,iRMS,LRMS,Fnonnat
0,DDI_predictor,run5_PF00004_PF10584_6EPF_I_resi222_resi355.C_...,run5,134,23,ranked_0,0.408763,74.897222,40.515000,58.717353,...,1044,672.0,134,NaN,NaN,None,0.024,10.611,36.414,1.0
1,DDI_predictor,run5_PF00004_PF10584_6EPF_I_resi222_resi355.C_...,run5,134,23,ranked_1,0.362116,76.172499,37.540001,63.295000,...,1044,672.0,134,NaN,NaN,None,0.021,9.798,40.902,1.0
2,DDI_predictor,run5_PF00004_PF10584_6EPF_I_resi222_resi355.C_...,run5,134,23,ranked_2,0.352951,78.841874,34.667273,60.844815,...,1044,672.0,134,NaN,NaN,None,0.024,10.028,36.797,1.0
3,DDI_predictor,run5_PF00004_PF10584_6EPF_I_resi222_resi355.C_...,run5,134,23,ranked_3,0.335142,79.413750,34.890001,58.636000,...,1044,672.0,134,NaN,NaN,None,0.019,11.070,42.954,1.0
4,DDI_predictor,run5_PF00004_PF10584_6EPF_I_resi222_resi355.C_...,run5,134,23,ranked_4,0.290458,74.833124,37.034167,58.633571,...,1044,672.0,134,NaN,NaN,None,0.025,10.426,35.423,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,DDI_predictor,run5_PF18773_PF00071_2X19_B_resi392_resi451.A_...,run5,60,169,ranked_0,0.731821,81.655001,86.288949,84.170573,...,1349,906.0,169,NaN,NaN,None,0.010,14.755,57.841,1.0
396,DDI_predictor,run5_PF18773_PF00071_2X19_B_resi392_resi451.A_...,run5,60,169,ranked_1,0.320172,60.186666,73.933501,68.042000,...,1349,906.0,169,NaN,NaN,None,0.010,14.636,59.054,1.0
397,DDI_predictor,run5_PF18773_PF00071_2X19_B_resi392_resi451.A_...,run5,60,169,ranked_2,0.261113,54.072222,87.745500,77.295172,...,1349,906.0,169,NaN,NaN,None,0.013,12.604,51.556,1.0
398,DDI_predictor,run5_PF18773_PF00071_2X19_B_resi392_resi451.A_...,run5,60,169,ranked_3,0.249534,55.004000,67.416922,60.767143,...,1349,906.0,169,NaN,NaN,None,0.011,14.383,56.060,1.0


In [1]:
# generate a column holding the respective PDB IDs the predcitions were based on to be able to merge them with
# df_Score_max
df_AF['PDB_ID'] = df_AF['prediction_name'].str.split('_').str[3]
df_AF['DDI_type'] = df_AF['prediction_name'].str.split('_').str[1] + '_' + df_AF['prediction_name'].str.split('_').str[2]

NameError: name 'df_AF' is not defined

In [48]:
df_AF

,project_name,prediction_name,run_id,chainA_length,chainB_length,model_id,model_confidence,chainA_intf_avg_plddt,chainB_intf_avg_plddt,intf_avg_plddt,...,num_align_resi_domain,RMSD_backbone_peptide,RMSD_all_atom_peptide,known_motif_plddt,DockQ,iRMS,LRMS,Fnonnat,PDB_ID,DDI_type
0,DDI_predictor,run5_PF00004_PF10584_6EPF_I_resi222_resi355.C_...,run5,134,23,ranked_0,0.408763,74.897222,40.515000,58.717353,...,134,NaN,NaN,None,0.024,10.611,36.414,1.0,6EPF,PF00004_PF10584
1,DDI_predictor,run5_PF00004_PF10584_6EPF_I_resi222_resi355.C_...,run5,134,23,ranked_1,0.362116,76.172499,37.540001,63.295000,...,134,NaN,NaN,None,0.021,9.798,40.902,1.0,6EPF,PF00004_PF10584
2,DDI_predictor,run5_PF00004_PF10584_6EPF_I_resi222_resi355.C_...,run5,134,23,ranked_2,0.352951,78.841874,34.667273,60.844815,...,134,NaN,NaN,None,0.024,10.028,36.797,1.0,6EPF,PF00004_PF10584
3,DDI_predictor,run5_PF00004_PF10584_6EPF_I_resi222_resi355.C_...,run5,134,23,ranked_3,0.335142,79.413750,34.890001,58.636000,...,134,NaN,NaN,None,0.019,11.070,42.954,1.0,6EPF,PF00004_PF10584
4,DDI_predictor,run5_PF00004_PF10584_6EPF_I_resi222_resi355.C_...,run5,134,23,ranked_4,0.290458,74.833124,37.034167,58.633571,...,134,NaN,NaN,None,0.025,10.426,35.423,1.0,6EPF,PF00004_PF10584
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,DDI_predictor,run5_PF18773_PF00071_2X19_B_resi392_resi451.A_...,run5,60,169,ranked_0,0.731821,81.655001,86.288949,84.170573,...,169,NaN,NaN,None,0.010,14.755,57.841,1.0,2X19,PF18773_PF00071
396,DDI_predictor,run5_PF18773_PF00071_2X19_B_resi392_resi451.A_...,run5,60,169,ranked_1,0.320172,60.186666,73.933501,68.042000,...,169,NaN,NaN,None,0.010,14.636,59.054,1.0,2X19,PF18773_PF00071
397,DDI_predictor,run5_PF18773_PF00071_2X19_B_resi392_resi451.A_...,run5,60,169,ranked_2,0.261113,54.072222,87.745500,77.295172,...,169,NaN,NaN,None,0.013,12.604,51.556,1.0,2X19,PF18773_PF00071
398,DDI_predictor,run5_PF18773_PF00071_2X19_B_resi392_resi451.A_...,run5,60,169,ranked_3,0.249534,55.004000,67.416922,60.767143,...,169,NaN,NaN,None,0.011,14.383,56.060,1.0,2X19,PF18773_PF00071


In [49]:
# Group df_AF by DDI type and derive a DDI type level feature for model confodence and DockQ by calculating the
# max value out of the 5 models per prediction of the highest scoring structure in 3did
df_AF_final_grouped_max_or = df_AF.groupby('DDI_type').agg({'model_confidence': max, 'DockQ': max})
df_AF_final_grouped_max_or = df_AF_final_grouped_max_or.rename(columns={'model_confidence': 'Max model confidence', 
                                                                        'DockQ': 'Max DockQ'})
df_AF_final_grouped_max_or.reset_index(inplace=True)

In [50]:
# Merge df_3did_HuRI_BioPlex_ProtCID with the newly generated Af DataFrame (df_AF_final_grouped_max_or) to 
# obtain all metrics aquired before as well as the AF metrics for the DDI types in the manual curation set
df_3did_HuRI_BioPlex_ProtCID_AF_mc = pd.merge(df_3did_HuRI_BioPlex_ProtCID, df_AF_final_grouped_max_or, on= 'DDI_type')
df_label = df_mc.groupby('DDI_type').agg({'DDI_approved': max})
df_3did_HuRI_BioPlex_ProtCID_AF_mc_ML = pd.merge(df_3did_HuRI_BioPlex_ProtCID_AF_mc, df_label.reset_index(), on= 'DDI_type')

In [51]:
df_3did_HuRI_BioPlex_ProtCID_AF_mc_ML

,DDI_type,DomainID1,DomainID2,DomainName1,DomainName2,Num_structures,Num_interchain_structures,Score,Zscore,contacts1,...,Num_intrachain_structures,HuRIzscore,BioPlexzscore,Num crystal forms,Min seq identity,Num of proteins,Num clusters,Max model confidence,Max DockQ,DDI_approved
0,PF00004_PF10584,PF00004,PF10584,AAA,Proteasome_A_N,1,1,3.28,2.97548,8,...,0,2.544618,-0.693878,6.0,56.0,5.0,3.0,0.408763,0.025,0.0
1,PF00009_PF01873,PF00009,PF01873,GTP_EFTU,eIF-5_eIF-2B,11,11,9.54,3.91978,32,...,0,6.517535,0.000000,1.0,86.0,1.0,4.0,0.205861,0.043,1.0
2,PF00010_PF02344,PF00010,PF02344,HLH,Myc-LZ,6,2,3.30,3.27272,16,...,4,0.811951,6.244998,5.0,85.0,3.0,4.0,0.578310,0.053,0.0
3,PF00019_PF00041,PF00019,PF00041,TGF_beta,fn3,1,1,4.56,2.98602,18,...,0,-1.412477,2.379070,1.0,100.0,1.0,0.0,0.177166,0.098,0.0
4,PF00023_PF07686,PF00023,PF07686,Ank,V-set,1,1,1.25,1.80728,7,...,0,0.182075,-2.004628,1.0,100.0,1.0,0.0,0.246959,0.017,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,PF16004_PF00400,PF16004,PF00400,EFTUD2,WD40,10,10,9.03,4.76148,24,...,0,2.599748,1.886777,33.0,13.0,6.0,2.0,0.882613,0.729,0.0
76,PF17292_PF08644,PF17292,PF08644,POB3_N,SPT16,3,3,2.26,2.49544,8,...,0,0.000000,18.230012,2.0,25.0,2.0,1.0,0.280585,0.145,0.0
77,PF17820_PF16523,PF17820,PF16523,PDZ_6,betaPIX_CC,1,1,3.24,2.33869,6,...,0,-0.153937,-0.153432,1.0,100.0,1.0,0.0,0.213756,0.005,0.0
78,PF17838_PF00071,PF17838,PF00071,PH_16,Ras,9,9,14.92,5.86783,27,...,0,10.363731,13.783072,1.0,23.0,6.0,1.0,0.889676,0.836,1.0


In [53]:
df_3did_HuRI_BioPlex_ProtCID_AF_mc_ML.to_csv('/Users/johgeist/Downloads/3did_project_dataframes_new/3did_Interactome_ProtCID_AF_metrics_mc_set.csv', index=False)

In [54]:
# Only keep those features from now on that should be provided to the Logistic Regression Model as the input
# Keep 'DDI_type' and 'DDI_approved' as index and label, too
df_3did_HuRI_BioPlex_ProtCID_AF_mc_ML = df_3did_HuRI_BioPlex_ProtCID_AF_mc_ML.filter(['DDI_type','Num_structures',
'Num_interchain_structures', 'Num_intrachain_structures', 'Score', 'Zscore', 'contacts1','Interchain_fraction',
'HuRIzscore', 'BioPlexzscore', 'Num crystal forms', 'Min seq identity', 'Num of proteins', 'Num clusters',
'Max model confidence', 'Max DockQ','DDI_approved'])

In [55]:
df_3did_HuRI_BioPlex_ProtCID_AF_mc_ML

,DDI_type,Num_structures,Num_interchain_structures,Num_intrachain_structures,Score,Zscore,contacts1,Interchain_fraction,HuRIzscore,BioPlexzscore,Num crystal forms,Min seq identity,Num of proteins,Num clusters,Max model confidence,Max DockQ,DDI_approved
0,PF00004_PF10584,1,1,0,3.28,2.97548,8,1.000000,2.544618,-0.693878,6.0,56.0,5.0,3.0,0.408763,0.025,0.0
1,PF00009_PF01873,11,11,0,9.54,3.91978,32,1.000000,6.517535,0.000000,1.0,86.0,1.0,4.0,0.205861,0.043,1.0
2,PF00010_PF02344,6,2,4,3.30,3.27272,16,0.333333,0.811951,6.244998,5.0,85.0,3.0,4.0,0.578310,0.053,0.0
3,PF00019_PF00041,1,1,0,4.56,2.98602,18,1.000000,-1.412477,2.379070,1.0,100.0,1.0,0.0,0.177166,0.098,0.0
4,PF00023_PF07686,1,1,0,1.25,1.80728,7,1.000000,0.182075,-2.004628,1.0,100.0,1.0,0.0,0.246959,0.017,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,PF16004_PF00400,10,10,0,9.03,4.76148,24,1.000000,2.599748,1.886777,33.0,13.0,6.0,2.0,0.882613,0.729,0.0
76,PF17292_PF08644,3,3,0,2.26,2.49544,8,1.000000,0.000000,18.230012,2.0,25.0,2.0,1.0,0.280585,0.145,0.0
77,PF17820_PF16523,1,1,0,3.24,2.33869,6,1.000000,-0.153937,-0.153432,1.0,100.0,1.0,0.0,0.213756,0.005,0.0
78,PF17838_PF00071,9,9,0,14.92,5.86783,27,1.000000,10.363731,13.783072,1.0,23.0,6.0,1.0,0.889676,0.836,1.0


In [56]:
# Load in the IUPred2a preduction results that were generated before as a DataFrame
df= pd.read_csv('/Users/johgeist/Downloads/IUPred_features_mc.csv', index_col=0)

In [58]:
df

,DDI_type,PDB_ID,Chain_ID1,DomainID1,DomainName1,DomainStart1,DomainEnd1,Chain_ID2,DomainID2,DomainName2,...,chain1_cryst_frac_iface_above04,chain2_cryst_frac_iface_above04,chain1_uniprot_avg_iface_iupred,chain2_uniprot_avg_iface_iupred,chain1_uniprot_frac_iface_above04,chain2_uniprot_frac_iface_above04,chain1_num_iface_res,chain2_num_iface_res,chain1_unmapped_iface_res,chain2_unmapped_iface_res
0,PF00004_PF10584,6EPF,I,PF00004,AAA,222,355,C,PF10584,Proteasome_A_N,...,0.400000,0.166667,0.235960,0.189883,0.000000,0.000000,5.0,6.0,0.0,0.0
1,PF00009_PF01873,2D74,A,PF00009,GTP_EFTU,9,201,B,PF01873,eIF-5_eIF-2B,...,0.000000,0.133333,0.170565,0.109300,0.000000,0.000000,17.0,15.0,0.0,0.0
2,PF00010_PF02344,6G6L,B,PF00010,HLH,214,254,C,PF02344,Myc-LZ,...,1.000000,0.571429,0.515222,0.102229,1.000000,0.000000,9.0,7.0,0.0,0.0
3,PF00019_PF00041,4UI2,B,PF00019,TGF_beta,294,395,A,PF00041,fn3,...,0.000000,0.100000,0.064760,0.292040,0.000000,0.000000,10.0,10.0,0.0,0.0
4,PF00023_PF07686,4NIK,A,PF00023,Ank,105,137,B,PF07686,V-set,...,0.666667,0.000000,0.377500,NaN,0.333333,NaN,3.0,7.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,PF16004_PF00400,6ID0,C,PF16004,EFTUD2,56,110,T,PF00400,WD40,...,0.900000,0.250000,0.501250,0.108217,0.900000,0.000000,10.0,12.0,0.0,0.0
76,PF17292_PF08644,4KHB,H,PF17292,POB3_N,5,97,G,PF08644,SPT16,...,0.000000,0.000000,0.173667,0.233467,0.000000,0.000000,6.0,3.0,0.0,0.0
77,PF17820_PF16523,3L4F,D,PF17820,PDZ_6,701,755,A,PF16523,betaPIX_CC,...,0.250000,1.000000,0.270425,0.791320,0.000000,1.000000,4.0,5.0,0.0,0.0
78,PF17838_PF00071,3KZ1,B,PF17838,PH_16,950,1079,E,PF00071,Ras,...,0.000000,0.166667,0.053487,0.179067,0.000000,0.166667,15.0,12.0,0.0,0.0


In [59]:
# Create two new columns holding the mean of the IUpred values from chain1 and 2 interface residues 
# or NaN if the IUPred of one of the two chains is not available (one column based on the crystallized
# sequence fragments, one on the full sequences)
df['uniprot_avg_iface_iupred'] = np.where((df['Uniprot_seq1'].isna() | df['Uniprot_seq2'].isna()), np.nan, df[['chain1_uniprot_avg_iface_iupred',
                                     'chain2_uniprot_avg_iface_iupred']].mean(axis=1))

df['cryst_avg_iface_iupred'] = df[['chain1_cryst_avg_iface_iupred',
                                     'chain2_cryst_avg_iface_iupred']].mean(axis=1)

In [60]:
# Only missing values for the full sequence based IUPreds
df[df['uniprot_avg_iface_iupred'].isna()]

,DDI_type,PDB_ID,Chain_ID1,DomainID1,DomainName1,DomainStart1,DomainEnd1,Chain_ID2,DomainID2,DomainName2,...,chain1_uniprot_avg_iface_iupred,chain2_uniprot_avg_iface_iupred,chain1_uniprot_frac_iface_above04,chain2_uniprot_frac_iface_above04,chain1_num_iface_res,chain2_num_iface_res,chain1_unmapped_iface_res,chain2_unmapped_iface_res,uniprot_avg_iface_iupred,cryst_avg_iface_iupred
4,PF00023_PF07686,4NIK,A,PF00023,Ank,105,137,B,PF07686,V-set,...,0.377500,NaN,0.333333,NaN,3.0,7.0,0.0,NaN,NaN,0.233657
5,PF00023_PF09067,6MOH,B,PF00023,Ank,109,141,C,PF09067,EpoR_lig-bind,...,NaN,0.13004,NaN,0.0,4.0,5.0,NaN,0.0,NaN,0.302410
13,PF00289_PF02436,5VYW,A,PF00289,Biotin_carb_N,1,110,D,PF02436,PYC_OADA,...,NaN,NaN,NaN,NaN,10.0,8.0,NaN,NaN,NaN,0.333180
17,PF00571_PF07686,5G5X,A,PF00571,CBS,235,289,B,PF07686,V-set,...,0.101271,NaN,0.000000,NaN,17.0,20.0,0.0,NaN,NaN,0.201863
36,PF04718_PF05405,6TT7,S,PF04718,ATP-synt_G,19,102,K,PF05405,Mt_ATP-synt_B,...,0.126727,NaN,0.181818,NaN,11.0,8.0,0.0,NaN,NaN,0.078319
50,PF07654_PF13855,6KA7,D,PF07654,C1-set,240,328,B,PF13855,LRR_8,...,0.250525,NaN,0.000000,NaN,4.0,5.0,0.0,NaN,NaN,0.202300
56,PF09291_PF07686,6FUR,A,PF09291,DUF1968,119,202,C,PF07686,V-set,...,NaN,NaN,NaN,NaN,12.0,12.0,NaN,NaN,NaN,0.322304


In [61]:
df_filtered = df.filter(['DDI_type', 'PDB_ID', 'uniprot_avg_iface_iupred', 'cryst_avg_iface_iupred'])

In [62]:
# For those cases where the IUPred of the full sequnece is missing, use the crystallized sequence based value
df_filtered['uniprot_avg_iface_iupred'].fillna(df_filtered['cryst_avg_iface_iupred'], inplace=True)

In [63]:
df_filtered.drop(columns=['PDB_ID'], inplace=True)

In [64]:
df_filtered

,DDI_type,uniprot_avg_iface_iupred,cryst_avg_iface_iupred
0,PF00004_PF10584,0.212922,0.313935
1,PF00009_PF01873,0.139932,0.126823
2,PF00010_PF02344,0.308725,0.485531
3,PF00019_PF00041,0.178400,0.158605
4,PF00023_PF07686,0.233657,0.233657
...,...,...,...
75,PF16004_PF00400,0.304733,0.383757
76,PF17292_PF08644,0.203567,0.175108
77,PF17820_PF16523,0.530872,0.609525
78,PF17838_PF00071,0.116277,0.110752


In [65]:
# Add the IUPred metrics to the df_3did_HuRI_BioPlex_ProtCID_AF_mc_ML DataFrame
df_3did_HuRI_BioPlex_ProtCID_AF_IUPred_mc_ML = pd.merge(df_3did_HuRI_BioPlex_ProtCID_AF_mc_ML, df_filtered,
                                                        on= 'DDI_type')

In [66]:
df_3did_HuRI_BioPlex_ProtCID_AF_IUPred_mc_ML

,DDI_type,Num_structures,Num_interchain_structures,Num_intrachain_structures,Score,Zscore,contacts1,Interchain_fraction,HuRIzscore,BioPlexzscore,Num crystal forms,Min seq identity,Num of proteins,Num clusters,Max model confidence,Max DockQ,DDI_approved,uniprot_avg_iface_iupred,cryst_avg_iface_iupred
0,PF00004_PF10584,1,1,0,3.28,2.97548,8,1.000000,2.544618,-0.693878,6.0,56.0,5.0,3.0,0.408763,0.025,0.0,0.212922,0.313935
1,PF00009_PF01873,11,11,0,9.54,3.91978,32,1.000000,6.517535,0.000000,1.0,86.0,1.0,4.0,0.205861,0.043,1.0,0.139932,0.126823
2,PF00010_PF02344,6,2,4,3.30,3.27272,16,0.333333,0.811951,6.244998,5.0,85.0,3.0,4.0,0.578310,0.053,0.0,0.308725,0.485531
3,PF00019_PF00041,1,1,0,4.56,2.98602,18,1.000000,-1.412477,2.379070,1.0,100.0,1.0,0.0,0.177166,0.098,0.0,0.178400,0.158605
4,PF00023_PF07686,1,1,0,1.25,1.80728,7,1.000000,0.182075,-2.004628,1.0,100.0,1.0,0.0,0.246959,0.017,0.0,0.233657,0.233657
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,PF16004_PF00400,10,10,0,9.03,4.76148,24,1.000000,2.599748,1.886777,33.0,13.0,6.0,2.0,0.882613,0.729,0.0,0.304733,0.383757
76,PF17292_PF08644,3,3,0,2.26,2.49544,8,1.000000,0.000000,18.230012,2.0,25.0,2.0,1.0,0.280585,0.145,0.0,0.203567,0.175108
77,PF17820_PF16523,1,1,0,3.24,2.33869,6,1.000000,-0.153937,-0.153432,1.0,100.0,1.0,0.0,0.213756,0.005,0.0,0.530872,0.609525
78,PF17838_PF00071,9,9,0,14.92,5.86783,27,1.000000,10.363731,13.783072,1.0,23.0,6.0,1.0,0.889676,0.836,1.0,0.116277,0.110752


In [68]:
df_3did_HuRI_BioPlex_ProtCID_AF_IUPred_mc_ML.to_csv('/Users/johgeist/Downloads/3did_project_dataframes_new/3did_Interactome_ProtCID_AF_IUPred_metrics_mc_set.csv', index=False)

In [69]:
# Rename columns and filter for those that will be used as features in the Logistic Regression Model
# Finished Training/ Test dataset
df_3did_HuRI_BioPlex_ProtCID_AF_IUPred_mc_ML.rename({'DDI_type': 'DDI type', 'Num_structures': 'Num structures',
        'Num_interchain_structures': 'Num interchain structures', 'Num_intrachain_structures': 'Num intrachain structures',
        'Score': '3did score', 'Zscore': '3did z-score', 'contacts1': 'Num res-res contacts',
        'Interchain_fraction': 'Fraction structures interchain',
        'HuRIzscore': 'HuRI z-score', 'BioPlexzscore': 'BioPlex z-score',
        'Num crystal forms': 'Num crystal forms in ProtCID cluster',
        'Min seq identity': 'Min seq identity in ProtCID cluster', 'Num of proteins': 'Num proteins in ProtCID cluster',
        'Max model confidence': 'Model confidence', 'Max DockQ': 'DockQ',
        'uniprot_avg_iface_iupred': 'Avg IUPred Iface full seq', 'cryst_avg_iface_iupred': 'Avg IUPred Iface cryst seq',
        'DDI_approved': 'DDI approved'},axis=1, inplace=True)

In [70]:
df_3did_HuRI_BioPlex_ProtCID_AF_IUPred_mc_ML = df_3did_HuRI_BioPlex_ProtCID_AF_IUPred_mc_ML.filter(['DDI type',
'Num structures', 'Num interchain structures', 'Num intrachain structures', '3did score', '3did z-score', 'Num res-res contacts', 'Fraction structures interchain', 
'HuRI z-score', 'BioPlex z-score', 'Num crystal forms in cluster', 
'Min seq identity in cluster', 'Num proteins in cluster', 'Model confidence', 'DockQ', 'Avg IUPred Iface full seq',
'Avg IUPred Iface cryst seq', 'DDI approved'])

In [71]:
df_3did_HuRI_BioPlex_ProtCID_AF_IUPred_mc_ML.to_csv('/Users/johgeist/Downloads/3did_project_dataframes_new/3did_Interactome_ProtCID_AF_IUPred_metrics_mc_set_for_ML.csv', index=False)

In [72]:
df_3did_HuRI_BioPlex_ProtCID_AF_IUPred_mc_ML.columns.tolist()

['DDI type',
 'Num structures',
 'Num interchain structures',
 'Num intrachain structures',
 '3did score',
 '3did z-score',
 'Num res-res contacts',
 'Fraction structures interchain',
 'HuRI z-score',
 'BioPlex z-score',
 'Num crystal forms in cluster',
 'Min seq identity in cluster',
 'Num proteins in cluster',
 'Model confidence',
 'DockQ',
 'Avg IUPred Iface full seq',
 'Avg IUPred Iface cryst seq',
 'DDI approved']

In [73]:
df_3did_HuRI_BioPlex_ProtCID_AF_IUPred_mc_ML

,DDI type,Num structures,Num interchain structures,Num intrachain structures,3did score,3did z-score,Num res-res contacts,Fraction structures interchain,HuRI z-score,BioPlex z-score,Num crystal forms in cluster,Min seq identity in cluster,Num proteins in cluster,Model confidence,DockQ,Avg IUPred Iface full seq,Avg IUPred Iface cryst seq,DDI approved
0,PF00004_PF10584,1,1,0,3.28,2.97548,8,1.000000,2.544618,-0.693878,6.0,56.0,5.0,0.408763,0.025,0.212922,0.313935,0.0
1,PF00009_PF01873,11,11,0,9.54,3.91978,32,1.000000,6.517535,0.000000,1.0,86.0,1.0,0.205861,0.043,0.139932,0.126823,1.0
2,PF00010_PF02344,6,2,4,3.30,3.27272,16,0.333333,0.811951,6.244998,5.0,85.0,3.0,0.578310,0.053,0.308725,0.485531,0.0
3,PF00019_PF00041,1,1,0,4.56,2.98602,18,1.000000,-1.412477,2.379070,1.0,100.0,1.0,0.177166,0.098,0.178400,0.158605,0.0
4,PF00023_PF07686,1,1,0,1.25,1.80728,7,1.000000,0.182075,-2.004628,1.0,100.0,1.0,0.246959,0.017,0.233657,0.233657,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,PF16004_PF00400,10,10,0,9.03,4.76148,24,1.000000,2.599748,1.886777,33.0,13.0,6.0,0.882613,0.729,0.304733,0.383757,0.0
76,PF17292_PF08644,3,3,0,2.26,2.49544,8,1.000000,0.000000,18.230012,2.0,25.0,2.0,0.280585,0.145,0.203567,0.175108,0.0
77,PF17820_PF16523,1,1,0,3.24,2.33869,6,1.000000,-0.153937,-0.153432,1.0,100.0,1.0,0.213756,0.005,0.530872,0.609525,0.0
78,PF17838_PF00071,9,9,0,14.92,5.86783,27,1.000000,10.363731,13.783072,1.0,23.0,6.0,0.889676,0.836,0.116277,0.110752,1.0


In [81]:
# Check for how many DDI types in the manual curation set there are missing values for the Interactome
# (HuRI/ BioPlex-cooccurrence) features -> cannot impute those as easy as the ProtCID metrics
len(df_3did_HuRI_BioPlex_ProtCID_AF_IUPred_mc_ML[(df_3did_HuRI_BioPlex_ProtCID_AF_IUPred_mc_ML['HuRI z-score'].isna())| (df_3did_HuRI_BioPlex_ProtCID_AF_IUPred_mc_ML['BioPlex z-score'].isna())])

16

In [2]:
# Filter those DDI types from the Traning/ Test dataset that were only supported by structures of 
# synthetic constructs
final_features_wsc = df_3did_HuRI_BioPlex_ProtCID_AF_IUPred_mc_ML.copy()
final_features_wsc['DDI type'] = final_features_wsc['DDI type'].str.split('_')
rows_sc = final_features_wsc.apply(lambda row: any(set(syn_cons_domains).intersection(row['DDI type'])), axis=1)
sc = final_features_wsc[rows_sc].copy()
sc['DDI type'] = sc['DDI type'].str.join('_')
sc_corrected = sc.drop(index = sc[sc['DDI type'].isin(['PF09291_PF07686', 'PF12796_PF00412'])].index)
sc_corrected_indices = sc_corrected.index.tolist()
final_features_wsc['DDI type'] = final_features_wsc['DDI type'].str.join('_')
final_features_wsc = final_features_wsc.drop(index = sc_corrected_indices)
final_features_wsc = final_features_wsc.drop(index=final_features_wsc[final_features_wsc['DDI type'] == 'PF07654_PF13855'].index)

NameError: name 'df_3did_HuRI_BioPlex_ProtCID_AF_IUPred_mc_ML' is not defined

In [83]:
#PF07654_PF13855 is synthetic construct without the sc domains
#PF09291_PF07686 is not a synthetic construct with the V-domain PF07686
#PF12796_PF00412 is not a synthetic construct including the PF12796 Ank2 domain

In [84]:
final_features_wsc

,DDI type,Num structures,Num interchain structures,Num intrachain structures,3did score,3did z-score,Num res-res contacts,Fraction structures interchain,HuRI z-score,BioPlex z-score,Num crystal forms in cluster,Min seq identity in cluster,Num proteins in cluster,Model confidence,DockQ,Avg IUPred Iface full seq,Avg IUPred Iface cryst seq,DDI approved
0,PF00004_PF10584,1,1,0,3.28,2.97548,8,1.000000,2.544618,-0.693878,6.0,56.0,5.0,0.408763,0.025,0.212922,0.313935,0.0
1,PF00009_PF01873,11,11,0,9.54,3.91978,32,1.000000,6.517535,0.000000,1.0,86.0,1.0,0.205861,0.043,0.139932,0.126823,1.0
2,PF00010_PF02344,6,2,4,3.30,3.27272,16,0.333333,0.811951,6.244998,5.0,85.0,3.0,0.578310,0.053,0.308725,0.485531,0.0
3,PF00019_PF00041,1,1,0,4.56,2.98602,18,1.000000,-1.412477,2.379070,1.0,100.0,1.0,0.177166,0.098,0.178400,0.158605,0.0
6,PF00026_PF06394,1,1,0,3.53,2.65889,27,1.000000,NaN,NaN,1.0,100.0,1.0,0.346743,0.079,0.290500,0.306567,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,PF16004_PF00400,10,10,0,9.03,4.76148,24,1.000000,2.599748,1.886777,33.0,13.0,6.0,0.882613,0.729,0.304733,0.383757,0.0
76,PF17292_PF08644,3,3,0,2.26,2.49544,8,1.000000,0.000000,18.230012,2.0,25.0,2.0,0.280585,0.145,0.203567,0.175108,0.0
77,PF17820_PF16523,1,1,0,3.24,2.33869,6,1.000000,-0.153937,-0.153432,1.0,100.0,1.0,0.213756,0.005,0.530872,0.609525,0.0
78,PF17838_PF00071,9,9,0,14.92,5.86783,27,1.000000,10.363731,13.783072,1.0,23.0,6.0,0.889676,0.836,0.116277,0.110752,1.0


In [94]:
print(final_features_wsc.columns)

Index(['DDI type', 'Num structures', 'Num interchain structures',
       'Num intrachain structures', '3did score', '3did z-score',
       'Num res-res contacts', 'Fraction structures interchain',
       'HuRI z-score', 'BioPlex z-score', 'Num crystal forms in cluster',
       'Min seq identity in cluster', 'Num proteins in cluster',
       'Model confidence', 'DockQ', 'Avg IUPred Iface full seq',
       'Avg IUPred Iface cryst seq', 'DDI approved'],
      dtype='object')


In [95]:
final_features_wsc.to_csv('/Users/johgeist/Downloads/3did_project_dataframes_new/3did_Interactome_ProtCID_AF_IUPred_metrics_mc_set_for_ML_without_synthetic_constructs.csv', index=False)